In [ ]:
from extract.extract import getTLEsFromLaunches, getCredentials
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import iqr
import pandas as pd
sns.set_theme()
sns.color_palette("Spectral", as_cmap=True)

start = datetime(2022, 1, 1)
end = datetime(2023, 1, 1)

token = getCredentials(source="discos")
username, password = getCredentials(source="spacetrack")

launchIDs = ["2013-066", "2018-092", "2019-084", "2022-002"]
#launchIDs = ["2019-084"]
#launchIDs = ["2013-066"]

# combining everything into one DF
ignoreList = [44827, 44828, 44829, 44830, 44831, 44832] + [39435, 39436, 39437] + [43704, 44046]
# combining everything into one DF
discosDataDict, dfOriginal = getTLEsFromLaunches(
    username,
    password,
    token,
    launchIDs,
    start,
    end,
    combineDiscosAndTLE=True,
    collectLaunches=True,
    collectAllTLEs=True,
    forceRegen=False,
    ignoreList=ignoreList
)

In [ ]:
print(dfOriginal.columns)
print(len(dfOriginal))
#df = dfOriginal.query("deltat< 10000").query("overallCovariance < 15")
df = dfOriginal.query("deltat>1").query("deltat< 150000").query("overallCovariance < 5").query("xSectAvg>0.00009999").query("BSTAR>0").query("BSTAR<0.007")
print(len(df))

colsToDrop = [
       'vimpelId', 'OBJECT_NAME', 'CENTER_NAME',
       'MEAN_MOTION', 'RA_OF_ASC_NODE',
       'ARG_OF_PERICENTER', 'MEAN_ANOMALY', 'ELEMENT_SET_NO',
       'REV_AT_EPOCH', 'MEAN_MOTION_DOT', 'MEAN_MOTION_DDOT',
       'SEMIMAJOR_AXIS', 'PERIOD', 'OBJECT_TYPE',
       'RCS_SIZE', 'COUNTRY_CODE', 'LAUNCH_DATE', 'DECAY_DATE', 'TLE_LINE0',
       'TLE_LINE1', 'TLE_LINE2', 'TLE_LINE1min1', 'TLE_LINE2min1']

df = df.drop(colsToDrop, axis=1)
print(df.columns)


In [ ]:
# corr = df.corr(numeric_only=True)
# fig, ax = plt.subplots(1,1, figsize=(15,15))
# sns.heatmap(corr, center=0, cmap="vlag", ax=ax)

In [ ]:
# sns.kdeplot(
#    data=df, x=df.overallCovariance, hue="name",
#    fill=True, common_norm=False, palette="crest",
#    alpha=.5, linewidth=0, legend=None
# )

In [ ]:
# sns.scatterplot(x=df.EPOCH, y=df.overallCovariance, hue=df.name)
# sns.rugplot(x=df.EPOCH, y=df.overallCovariance, height=.1, hue=df.name, legend=None)

In [ ]:
# sns.scatterplot(x=df.deltat, y=df.overallCovariance, hue=df.name, legend=None)
# sns.rugplot(x=df.deltat, y=df.overallCovariance, height=.1, hue=df.name, legend=None)

In [ ]:
# plot = sns.displot(x=(df.deltat), y=df.overallCovariance, kind="kde")


In [ ]:
#plot = sns.boxenplot(y=df.name, x=(df.deltat))


In [ ]:
# sns.boxenplot(
#     x=df['shape'], y=df.overallCovariance,
# )

# print(df['shape'].unique())
# df.query('shape=="Box"')['shape']
# df.groupby("shape").size()

In [ ]:
#print(df.groupby("name").mean()["deltat"])


# #sns.scatterplot(x=df.deltat, y=df.errorAlong, hue=df.name,legend=None)
# #sns.scatterplot(x=df.deltat, y=df.errorCross, hue=df.name,legend=None)
# sns.scatterplot(x=df.deltat, y=df.errorRadial, hue=df.name,legend=None)


In [ ]:
#df.groupby("name").min()["ECCENTRICITY"]
#print(df.groupby("name").max()["mass"])
df = df.dropna(subset = ['width'])


d = df["name"].count()
print(d)


# max is 1.8368 and min 0.00625
#print(df.groupby('shape').size())
# a = df[df['shape']=='Hex Cyl + 3 Pan'].index.values
# df.iloc[a]
#width', 'xSectMax',
#       'height', 'diameter'APOAPSIS
# print(np.size(df.mass))
# a = df.groupby("name")query("mass< 5")
# print(a)
#sns.scatterplot(x=df.mass, y=np.size(df.mass), hue=df.name ,legend=None)


In [ ]:
# MASS 
# import time

# df['datetime'] = pd.to_datetime(df['EPOCH'], unit='s')
# df = df.sort_values(by='datetime')
#dat = df.groupby('NORAD_CAT_ID').size().plot(kind = "bar") #bar of all the norad and how manyb points

m = df.groupby("name").max()["mass"]
#sns.barplot(x = m.index, y = m.values)


pico = m[(m < 1.1)].count()
nano = m[(m > 1) & (m < 11)].count()
micro = m[(m > 10) & (m < 101)].count()
mini = m[(m > 100) & (m < 500)].count()




fig, ax = plt.subplots(1,1, figsize = (7,5))
sns.barplot(x=[pico,nano,micro,mini], y=["Pico (0-1kg)","Nano (1-10kg)","Micro (10-100kg)","Mini (100-300kg)"]).set(title='Mass distribution of dataframe',xlabel ="Number of Satellites")
fig.set_tight_layout(True)
fig.savefig("data/mass_quantity.png")


picodf = df.query("mass < 1.01")
nanodf = df.query("mass > 1").query("mass < 11")
microdf = df.query("mass > 10").query("mass < 101")
minidf = df.query("mass > 100").query("mass < 500")


#pico_df = m.query("mass < 2")




In [ ]:
# MASS 
#sns.barplot( x=[pico.count(),nano.count(),micro.count(),mini.count()], y=["Pico (0-1kg)","Nano (1-10kg)","Micro (10-100kg)","Mini (100-300kg)"],hue=None)

picom = picodf["mass"] #pico 0-1
nanom = nanodf["mass"] # nano 2-10
microm = microdf["mass"] #micro 10-100
minim = minidf["mass"] #mini 0-1


fig, ax = plt.subplots(1,1, figsize = (7,5))
plot = sns.barplot(x=[picom.count(),nanom.count(),microm.count(),minim.count()], y=["Pico (0-1kg)","Nano (1-10kg)","Micro (10-100kg)","Mini (100-300kg)"], ax = ax)
fig.set_tight_layout(True)
fig.savefig("data/mass_quantityoftles.png")

In [ ]:
# fig, ax = plt.subplots(4,1, figsize = (7,15))
# sns.scatterplot(x=picodf.overallCovariance, y=picodf.ECCENTRICITY, ax= ax[0])
# sns.scatterplot(x=nanodf.overallCovariance, y=nanodf.ECCENTRICITY, ax= ax[1])
# sns.scatterplot(x=microdf.overallCovariance, y=microdf.ECCENTRICITY, ax= ax[2])
# sns.scatterplot(x=minidf.overallCovariance, y=minidf.ECCENTRICITY, ax= ax[3])
# fig.set_tight_layout(True)
# fig.savefig("data/eccen_covariance.png")

In [ ]:
# fig, ax = plt.subplots(4,1, figsize = (7,15))
# sns.scatterplot(x=picodf.overallCovariance, y=picodf.ECCENTRICITY, style=picodf.NORAD_CAT_ID, hue=picodf.EPOCH,palette='coolwarm', legend=None, ax=ax[0])
# sns.scatterplot(x=nanodf.overallCovariance, y=nanodf.ECCENTRICITY, style=nanodf.NORAD_CAT_ID, hue=nanodf.EPOCH,palette='coolwarm', legend=None, ax= ax[1])
# sns.scatterplot(x=microdf.overallCovariance, y=microdf.ECCENTRICITY, style=microdf.NORAD_CAT_ID, hue=microdf.EPOCH,palette='coolwarm', legend=None, ax=ax[2])
# sns.scatterplot(x=minidf.overallCovariance, y=minidf.ECCENTRICITY, style=minidf.NORAD_CAT_ID, hue=minidf.EPOCH,palette='Blues', legend=None, ax= ax[3])
# fig.set_tight_layout(True)
# fig.savefig("data/eccen_covariance_wcolour.png")





In [ ]:
# MASS 
cap = (minidf.EPOCH)
bob = ((minidf.actualX)**2 +(minidf.actualY)**2+(minidf.actualZ)**2)**(1/2)
# df.drop(df.NORAD_CAT_ID == 51030)
sns.lineplot(x=cap, y=bob, style=minidf.NORAD_CAT_ID, hue=minidf.NORAD_CAT_ID )


In [ ]:
# VOLUME 
vol = df.height*df.depth*df.width
df['Vol'] = vol

Alt = np.sqrt((df.actualX)**2 +(df.actualY)**2+(df.actualZ)**2)
df['Alt'] = Alt

df['xSectAvg'].mean()


In [ ]:

v = df.groupby("name").max()["Vol"]
# print(v.count())

# sns.scatterplot(x=df.overallCovariance, y=df.Vol, hue=df.NORAD_CAT_ID, palette='rainbow')

pocket = v[(v < 0.0001250000001)].count()
cube1 = v[(v > 0.0001251) & (v < 0.001000001)].count() #cube 2u and 6u
cube2 = v[(v > 0.0011) & (v < 0.00200001)].count()
cube3 = v[(v > 0.0021) & (v < 0.003000001)].count()
cube6 = v[(v > 0.0031) & (v < 0.006000001)].count()
cube12 = v[(v > 0.0061) & (v < 0.012000001)].count()
cube16 = v[(v > 0.0121) & (v < 0.016000001)].count()
cube27 = v[(v > 0.0161) & (v < 0.02700001)].count()
nano1 = v[(v > 0.0271) & (v < 0.200001)].count()
nano2 = v[(v > 0.21) & (v < 0.40001)].count()
nano3 = v[(v > 0.41) & (v < 1.000001)].count()


print(pocket,cube1,cube2,cube3,cube6,cube12,cube16,cube27,nano1,nano2,nano3)


# sum = pocket+cube1+cube2+cube3+cube6+cube12+cube16+cube27+nano1+nano2+nano3
# print(sum)

fig, ax = plt.subplots(1,1,figsize = (7,5))
sns.barplot(x=[pocket,cube1,cube2,cube3,cube6,cube12,cube16,cube27,nano1,nano2,nano3], y=["Pocketcube","Cube 1U","Cube 2U","Cube 3U","Cube 6U","Cube 12U","Cube 16U","Cube 27U",r"$0.027m^3 - 0.2m^3$",r"$0.2m^3 - 0.4 m^3$",r'$0.4m^3 - 1m^3$']).set(title='Volume distribution of data',xlabel ="Number of Satellites")
fig.set_tight_layout(True)
fig.savefig("data/sat_volume_quantity.png")
# 110037


In [ ]:
# VOLUME 

# pocketdf = df.query("volume < 0.0001250000001")
# cube1df = df.query("volume > 0.0001251").query("volume < 0.001000001")
# cube2df = df.query("volume > 0.0011").query("volume <0.00200001")
# cube3df = df.query("volume > 0.0021").query("volume <0.00300001")
# cube6df = df.query("volume > 0.0031").query("volume <0.00600001")
# cube12df = df.query("volume > 0.0061").query("volume <0.01200001")
# cube16df = df.query("volume > 0.0121").query("volume <1")


smallvdf = df.query("volume < 0.00200001")
cube3df = df.query("volume > 0.0021").query("volume <0.00300001")
bigvdf = df.query("volume > 0.0031").query("volume <1")







In [ ]:
# VOLUME 
fig, ax = plt.subplots(1,3,sharey=True,figsize = (14,5))
fig.suptitle('Eccentricity vs Covariance')
ax[0].set_title(r'$V = 0-0.003m^3$')
sns.scatterplot(x=smallvdf.overallCovariance, y=smallvdf.ECCENTRICITY, hue=smallvdf.NORAD_CAT_ID, palette='Spectral',legend=None, ax= ax[0])
ax[1].set_title(r'$V = 0.003-0.004m^3$')
sns.scatterplot(x=cube3df.overallCovariance, y=cube3df.ECCENTRICITY, hue=cube3df.NORAD_CAT_ID, palette='Spectral', legend=None,ax= ax[1])
ax[2].set_title(r'$V = 0.004-1m^3$')
sns.scatterplot(x=bigvdf.overallCovariance, y=bigvdf.ECCENTRICITY, hue=bigvdf.NORAD_CAT_ID,palette='Spectral', legend=None,ax= ax[2])
ax[0].set(xlabel='Overall Covariance [km]', ylabel='Eccentricity [-]')
ax[1].set(xlabel='Overall Covariance [km]', ylabel='Eccentricity [-]')
ax[2].set(xlabel='Overall Covariance [km]', ylabel='Eccentricity [-]')

fig.savefig("data/eccen_covariance_vol.png")




In [ ]:
# # VOLUME 
# fig, ax = plt.subplots(3,1,sharex=True,figsize = (5,7))
# sns.lineplot(x=smallvdf.EPOCH, y=(((smallvdf.actualX)**2 +(smallvdf.actualY)**2+(smallvdf.actualZ)**2)**(1/2)), style=smallvdf.NORAD_CAT_ID, hue=smallvdf.NORAD_CAT_ID, legend=None, ax= ax[0])
# sns.lineplot(x=cube3df.EPOCH, y=(((cube3df.actualX)**2 +(cube3df.actualY)**2+(cube3df.actualZ)**2)**(1/2)), style=cube3df.NORAD_CAT_ID, hue=cube3df.NORAD_CAT_ID, legend=None, ax= ax[1])
# sns.lineplot(x=bigvdf.EPOCH, y=(((bigvdf.actualX)**2 +(bigvdf.actualY)**2+(bigvdf.actualZ)**2)**(1/2)), style=bigvdf.NORAD_CAT_ID, hue=bigvdf.NORAD_CAT_ID, legend=None, ax= ax[2])
# fig.set_tight_layout(True)
# fig.savefig("data/Altitude_epoch_volume.png")

In [ ]:
# SURFACE AREA 


A = df.groupby("name").max()["xSectAvg"]
print(A.count())





# sns.scatterplot(x=df.overallCovariance, y=df.xSectAvg, hue=df.NORAD_CAT_ID, palette='Spectral')

surface1 = A[(A < 0.1)].count()
surface2 = A[(A > 0.1000001) & (A < 0.5)].count()
surface3 = A[(A > 0.5000001) & (A < 2.0)].count()


print(surface1,surface2,surface3)


sums = surface1+surface2+surface3
print(sums)

subssurface1 = A[(A < 0.015)].count()
subssurface2 = A[(A > 0.014) & (A < 0.075)].count()
subssurface3 = A[(A > 0.074) & (A < 0.1)].count()
print(subssurface1,subssurface2,subssurface3)


sumss = subssurface1+subssurface2+subssurface3
print(sumss)

fig, ax = plt.subplots(1,1,figsize = (7,5))
sns.barplot(x=[subssurface1, subssurface2, subssurface3,surface2,surface3], y=[r"$0-0.015m^2$",r"$0.015-0.075m^2$",r"$0.075-0.1m^2$",r"$0.1-0.5m^2$",r"$0.5-2.0m^2$"]).set(title='Average surface area distribution of data',xlabel ="Number of Satellites",ylabel = r"Surface Area $[m^2]$")
fig.set_tight_layout(True)
fig.savefig("data/sat_xsectav_quantity.png")


In [ ]:
# SURFACE AREA 


surface1df = df.query("xSectAvg < 0.099999")
surface2df = df.query("xSectAvg > 0.1000001").query("xSectAvg < 0.5")
surface3df = df.query("xSectAvg > 0.5000001").query("xSectAvg < 2.8")



In [ ]:


fig, ax = plt.subplots(1,3,sharey=True,figsize = (13,4))
fig.suptitle('Eccentricity vs Covariance')
ax[0].set_title(r'Surface area = 0-0.1$m^2$')
sns.scatterplot(x=surface1df.overallCovariance, y=surface1df.ECCENTRICITY, hue=surface1df.NORAD_CAT_ID, style =surface1df.NORAD_CAT_ID, palette='Spectral',legend=None, ax= ax[0])
ax[1].set_title(r'Surface area = 0.1-0.5$m^2$')
sns.scatterplot(x=surface2df.overallCovariance, y=surface2df.ECCENTRICITY, hue=surface2df.NORAD_CAT_ID, style =surface2df.NORAD_CAT_ID, palette='Spectral', legend=None,ax= ax[1])
ax[2].set_title(r'Surface area = 0.5-2.7$m^2$')
sns.scatterplot(x=surface3df.overallCovariance, y=surface3df.ECCENTRICITY, hue=surface3df.NORAD_CAT_ID,style =surface3df.NORAD_CAT_ID,palette='Spectral', legend=None,ax= ax[2])
ax[0].set(xlabel='Overall Covariance [km]', ylabel='Eccentricity [-]')
ax[1].set(xlabel='Overall Covariance [km]', ylabel='Eccentricity [-]')
ax[2].set(xlabel='Overall Covariance [km]', ylabel='Eccentricity [-]')
fig.savefig("data/eccen_covariance_surfaceav.png")


In [ ]:

subsurface1df = surface1df.query("xSectAvg < 0.015")
subsurface2df = surface1df.query("xSectAvg > 0.014").query("xSectAvg < 0.075")
subsurface3df = surface1df.query("xSectAvg > 0.074").query("xSectAvg < 0.1")

# fig, ax = plt.subplots(1,3,sharey=True,figsize = (13,4))
# fig.suptitle('Eccentricity vs Covariance')
# ax[0].set_title(r'Surface area = 0-0.015$m^2$')
# sns.scatterplot(x=subsurface1df.overallCovariance, y=subsurface1df.ECCENTRICITY, hue=subsurface1df.NORAD_CAT_ID, style =subsurface1df.NORAD_CAT_ID, palette='Spectral',legend=None, ax= ax[0])
# ax[1].set_title(r'Surface area = 0.015-0.075$m^2$')
# sns.scatterplot(x=subsurface2df.overallCovariance, y=subsurface2df.ECCENTRICITY, hue=subsurface2df.NORAD_CAT_ID, style =subsurface2df.NORAD_CAT_ID, palette='Spectral', legend=None,ax= ax[1])
# ax[2].set_title(r'Surface area = 0.075-0.1$m^2$')
# sns.scatterplot(x=subsurface3df.overallCovariance, y=subsurface3df.ECCENTRICITY, hue=subsurface3df.NORAD_CAT_ID,style =subsurface3df.NORAD_CAT_ID,palette='Spectral', legend=None,ax= ax[2])
# ax[0].set(xlabel='Overall Covariance [km]', ylabel='Eccentricity [-]')
# ax[1].set(xlabel='Overall Covariance [km]', ylabel='Eccentricity [-]')
# ax[2].set(xlabel='Overall Covariance [km]', ylabel='Eccentricity [-]')
# fig.savefig("data/eccen_covariance_surfaceav_butsub0.1m2.png")



In [ ]:
# SURFACE 
# fig, ax = plt.subplots(3,1,sharex=True,figsize = (5,7))
# sns.lineplot(x=surface1df.EPOCH, y=(((surface1df.actualX)**2 +(surface1df.actualY)**2+(surface1df.actualZ)**2)**(1/2)), style=surface1df.NORAD_CAT_ID, hue=surface1df.NORAD_CAT_ID, legend=None, ax= ax[0])
# sns.lineplot(x=surface2df.EPOCH, y=(((surface2df.actualX)**2 +(surface2df.actualY)**2+(surface2df.actualZ)**2)**(1/2)), style=surface2df.NORAD_CAT_ID, hue=surface2df.NORAD_CAT_ID, legend=None, ax= ax[1])
# sns.lineplot(x=surface3df.EPOCH, y=(((surface3df.actualX)**2 +(surface3df.actualY)**2+(surface3df.actualZ)**2)**(1/2)), style=surface3df.NORAD_CAT_ID, hue=surface3df.NORAD_CAT_ID, legend=None, ax= ax[2])
# fig.set_tight_layout(True)
# # fig.savefig("data/Altitude_epoch_surface.png")

# fig, ax = plt.subplots(3,1,sharex=True,figsize = (5,7))
# sns.lineplot(x=subsurface1df.EPOCH, y=(((subsurface1df.actualX)**2 +(subsurface1df.actualY)**2+(subsurface1df.actualZ)**2)**(1/2)), style=subsurface1df.NORAD_CAT_ID, hue=subsurface1df.NORAD_CAT_ID, legend=None, ax= ax[0])
# sns.lineplot(x=subsurface2df.EPOCH, y=(((subsurface2df.actualX)**2 +(subsurface2df.actualY)**2+(subsurface2df.actualZ)**2)**(1/2)), style=subsurface2df.NORAD_CAT_ID, hue=subsurface2df.NORAD_CAT_ID, legend=None, ax= ax[1])
# sns.lineplot(x=subsurface3df.EPOCH, y=(((subsurface3df.actualX)**2 +(subsurface3df.actualY)**2+(subsurface3df.actualZ)**2)**(1/2)), style=subsurface3df.NORAD_CAT_ID, hue=subsurface3df.NORAD_CAT_ID, legend=None, ax= ax[2])
# fig.set_tight_layout(True)
# fig.savefig("data/Altitude_epoch_subsurface.png")

In [ ]:
fig, ax = plt.subplots(1,1,sharex=True,figsize = (7,5))
fig.suptitle('Altitude vs Atmospheric Drag Term')
sns.scatterplot(x=df.BSTAR, y=(((df.actualX)**2 +(df.actualY)**2+(df.actualZ)**2)**(1/2)), hue=df.NORAD_CAT_ID, legend=None, ax=ax)
ax.set(xlabel=r'Atmospheric Drag Term [$1/r_e$]', ylabel='Altitude [m]')
fig.savefig("data/BSTAR_Alt_all.png")  

In [ ]:
sns.lineplot(x=df.EPOCH, y=(((df.actualX)**2 +(df.actualY)**2+(df.actualZ)**2)**(1/2)), style=df.NORAD_CAT_ID, hue=df.NORAD_CAT_ID, legend=None)

In [ ]:
#NORAD ALTITUDE
Alt = np.sqrt((df.actualX)**2 +(df.actualY)**2+(df.actualZ)**2)
df['Alt'] = Alt

H = df.groupby("name").max()["NORAD_CAT_ID"]
print(H.count())





# sns.scatterplot(x=df.overallCovariance, y=df.xSectAvg, hue=df.NORAD_CAT_ID, palette='Spectral')

Alt1 = H[(H < 40000)].count()
Alt2 = H[(H > 40000) & (H < 46000)].count()
Alt3 = H[(H > 46000) & (H < 52000)].count()


print(Alt1,Alt2,Alt3)


suma = Alt1+Alt2+Alt3
print(suma)


fig, ax = plt.subplots(1,1,figsize = (7,5))
sns.barplot(x=[Alt1, Alt2, Alt3], y=["<40000","40000-46000","46000-52000"]).set(title='Average norad distribution of data',xlabel ="Number of Satellites",ylabel = r"Altitude $[m]$")
fig.set_tight_layout(True)
fig.savefig("data/sat_Norad_quantity.png")

alt1df = df.query("NORAD_CAT_ID < 40000")
alt2df = df.query("NORAD_CAT_ID > 40000").query("NORAD_CAT_ID < 46000")
alt3df = df.query("NORAD_CAT_ID > 46000").query("NORAD_CAT_ID < 52000")



In [ ]:
fig, ax = plt.subplots(1,3,sharey=True,figsize = (13,4))
fig.suptitle('Altitude vs Atmospheric Drag Term')
ax[0].set_title(r'Surface area = 0-0.1$m^2$')
sns.scatterplot(x=surface1df.BSTAR, y=surface1df.Alt, hue=surface1df.NORAD_CAT_ID, style =surface1df.NORAD_CAT_ID, palette='Spectral',legend=None, ax= ax[0])
ax[1].set_title(r'Surface area = 0.1-0.5$m^2$')
sns.scatterplot(x=surface2df.BSTAR, y=surface2df.Alt, hue=surface2df.NORAD_CAT_ID, style =surface2df.NORAD_CAT_ID, palette='Spectral', legend=None,ax= ax[1])
ax[2].set_title(r'Surface area = 0.5-2.7$m^2$')
sns.scatterplot(x=surface3df.BSTAR, y=surface3df.Alt, hue=surface3df.NORAD_CAT_ID,style =surface3df.NORAD_CAT_ID,palette='Spectral', legend=None,ax= ax[2])
ax[0].set(xlabel='BSTAR', ylabel='alti')
ax[1].set(xlabel='BSTAR', ylabel='alt [-]')
ax[2].set(xlabel='BSTAR', ylabel='alt [-]')
fig.savefig("data/BSTAR_covariance_surfaceav.png")

In [ ]:
alt3df.loc[df['NORAD_CAT_ID']==51080].Alt

In [ ]:
fig, ax = plt.subplots(1,3,figsize = (15,5))
fig.suptitle('Altitude vs Time')
ax[0].set_title('norad 30000-40000')
sns.lineplot(x=alt1df.EPOCH, y=alt1df.Alt, hue=alt1df.NORAD_CAT_ID, style =alt1df.NORAD_CAT_ID, palette='Spectral',legend=None, ax= ax[2])
ax[1].set_title('norad 40000-46000')
sns.lineplot(x=alt2df.EPOCH, y=alt2df.Alt, hue=alt2df.NORAD_CAT_ID, style =alt2df.NORAD_CAT_ID, palette='Spectral',legend=None,ax= ax[0])
ax[2].set_title('norad 46000 - 52000')
sns.lineplot(x=alt3df.EPOCH, y=alt3df.Alt, hue=alt3df.NORAD_CAT_ID,style =alt3df.NORAD_CAT_ID,palette='Spectral',legend=None,ax= ax[1])
ax[0].set(xlabel='epoch [-]', ylabel='alt [m]')
ax[0].set_xticklabels(ax[0].get_xticklabels(), rotation=40, ha="right")
ax[1].set(xlabel='epoch [-]', ylabel='alt [m]')
ax[1].set_xticklabels(ax[1].get_xticklabels(), rotation=40, ha="right")
ax[2].set(xlabel='epoch [-]', ylabel='alt [m]')
ax[2].set_xticklabels(ax[2].get_xticklabels(), rotation=40, ha="right")
fig.savefig("data/alt_time_norad.png")

In [ ]:
fig, ax = plt.subplots(1,3,figsize = (15,5))
fig.suptitle('Altitude vs Time')
ax[0].set_title('norad 30000-40000')
sns.scatterplot(x=alt1df.NORAD_CAT_ID, y=alt1df.deltat, style =alt1df.NORAD_CAT_ID, palette='Spectral', ax= ax[2])
ax[1].set_title('norad 40000-46000')
sns.scatterplot(x=alt2df.NORAD_CAT_ID, y=alt2df.deltat, style =alt2df.NORAD_CAT_ID, palette='Spectral',ax= ax[0])
ax[2].set_title('norad 46000 - 52000')
sns.scatterplot(x=alt3df.NORAD_CAT_ID, y=alt3df.deltat, style =alt3df.NORAD_CAT_ID,palette='Spectral',ax= ax[1])
ax[0].set(xlabel='epoch [-]', ylabel='alt [m]')
ax[0].set_xticklabels(ax[0].get_xticklabels(), rotation=40, ha="right")
ax[1].set(xlabel='epoch [-]', ylabel='alt [m]')
ax[1].set_xticklabels(ax[1].get_xticklabels(), rotation=40, ha="right")
ax[2].set(xlabel='epoch [-]', ylabel='alt [m]')
ax[2].set_xticklabels(ax[2].get_xticklabels(), rotation=40, ha="right")
fig.savefig("data/alt_deltat_norad.png")

In [ ]:
# fig, ax = plt.subplots(1,3,figsize = (13,5))
# fig.suptitle('Altitude vs Time')
# ax[0].set_title('norad 30000-40000')
# sns.scatterplot(x=alt1df.BSTAR, y=alt1df.Alt, hue=alt1df.xSectAvg, style =alt1df.NORAD_CAT_ID, palette='Spectral',legend=None, ax= ax[2])
# ax[1].set_title('norad 40000-46000')
# sns.scatterplot(x=alt2df.BSTAR, y=alt2df.Alt, hue=alt2df.xSectAvg, style =alt2df.NORAD_CAT_ID, palette='Spectral',legend=None,ax= ax[0])
# ax[2].set_title('norad 46000 - 52000')
# sns.scatterplot(x=alt3df.BSTAR, y=alt3df.Alt, hue=alt3df.xSectAvg,style =alt3df.NORAD_CAT_ID,palette='Spectral',legend=None,ax= ax[1])
# ax[0].set(xlabel=r'Atmospheric Drag Term [$1/r_e$]', ylabel='alt [m]')
# ax[1].set(xlabel=r'Atmospheric Drag Term [$1/r_e$]', ylabel='alt [m]')
# ax[2].set(xlabel=r'Atmospheric Drag Term [$1/r_e$]', ylabel='alt [m]')
# fig.savefig("data/Bstar_alt_norad.png")

In [ ]:
fig, ax = plt.subplots(1,2, figsize = (15,5))
sns.barplot(x=[surface1,surface2,surface3], y=["0-0.1","0.1-0.5","0.5-2.0"],ax = ax[0]).set(title='(a) Average cross sectional area distribution',xlabel ="Number of Satellites",ylabel = r"Average Cross sectional Area $[m^2]$")
sns.barplot(x=[(pocket+cube1+cube2),cube3,(cube6+cube12+cube16+cube27+nano1+nano2+nano3)], y=["Pocketcube to 2U","Cube 3U",r"6U-1$m^3$"], ax = ax[1]).set(title='(b) Volume distribution of data',xlabel ="Number of Satellites")
fig.set_tight_layout(True)
fig.savefig("data/mass_surface_volume_2quantity.png")

In [ ]:
# sns.scatterplot(x=smallvdf.overallCovariance, y=smallvdf.deltat,hue = smallvdf.NORAD_CAT_ID,palette='Spectral')
# plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)

fig, ax = plt.subplots(1,3,figsize = (15,5))
fig.suptitle('Delta t  vs Overall Covariance ')
ax[0].set_title(r'Volume <0.002 $m^3$')
sns.scatterplot(x=smallvdf.overallCovariance, y=smallvdf.deltat, hue =smallvdf.NORAD_CAT_ID, palette='Spectral',legend=None, ax= ax[0])
ax[1].set_title(r'Volume 0.002-0.003 $m^3$')
sns.scatterplot(x=cube3df.overallCovariance, y=cube3df.deltat, hue =cube3df.NORAD_CAT_ID, palette='Spectral',legend=None,ax= ax[1])
ax[2].set_title(r'Volume 0.003-1 $m^3$')
sns.scatterplot(x=bigvdf.overallCovariance, y=bigvdf.deltat, hue =bigvdf.NORAD_CAT_ID,palette='Spectral',legend=None,ax= ax[2])
ax[0].set(xlabel='overall convarience [km]', ylabel='Deltat [sec]')
ax[1].set(xlabel='overall convarience [km]', ylabel='Deltat [sec]')
ax[2].set(xlabel='overall convarience [km]', ylabel='Deltat [sec]')
fig.savefig("data/con_deltat_noradbutsurface.png")

# B = df.groupby("name").min()["deltat"]
# print(B)





In [ ]:
z = surface1df.groupby("name").mean()["deltat"]
i = surface1df.groupby("name").max()["overallCovariance"]

sns.scatterplot(x=z, y =i)
# A = df.groupby("deltat").mean()["deltat"]



In [ ]:
# sns.lineplot(x=alt3df.loc[df['NORAD_CAT_ID']==51082].EPOCH, y=alt3df.loc[df['NORAD_CAT_ID']==51082].Alt, palette='Spectral')
#sns.lineplot(x=alt3df.loc[df['NORAD_CAT_ID']==51083].EPOCH, y=alt3df.loc[df['NORAD_CAT_ID']==51083].Alt, palette='Spectral')
# sns.lineplot(x=alt3df.loc[df['NORAD_CAT_ID']==51085].EPOCH, y=alt3df.loc[df['NORAD_CAT_ID']==51085].Alt, palette='Spectral')
#sns.lineplot(x=alt3df.loc[df['NORAD_CAT_ID']==51086].EPOCH, y=alt3df.loc[df['NORAD_CAT_ID']==51086].Alt, palette='Spectral')
# sns.lineplot(x=alt3df.loc[df['NORAD_CAT_ID']==51087].EPOCH, y=alt3df.loc[df['NORAD_CAT_ID']==51087].Alt, palette='Spectral')
# sns.lineplot(x=alt3df.loc[df['NORAD_CAT_ID']==51088].EPOCH, y=alt3df.loc[df['NORAD_CAT_ID']==51088].Alt, palette='Spectral')
sns.lineplot(x=alt2df.loc[df['NORAD_CAT_ID']==44041].EPOCH.iloc[0:100], y=alt2df.loc[df['NORAD_CAT_ID']==44041].Alt.iloc[0:100], palette='Spectral')






In [ ]:
sns.scatterplot(x=alt2df.loc[df['NORAD_CAT_ID']==44041].EPOCH, y=alt2df.loc[df['NORAD_CAT_ID']==44041].deltat, palette='Spectral')


In [ ]:
fig, ax = plt.subplots(1,3,figsize = (15,5))
fig.suptitle(' Altitude  vs Overall Covariance ')
ax[0].set_title(r'Volume <0.002 $m^2$')
sns.scatterplot(x=smallvdf.errorAlong, y=smallvdf.BSTAR, hue =smallvdf.NORAD_CAT_ID, palette='Spectral',legend=None, ax= ax[2])
ax[1].set_title(r'Volume 0.002-0.003 $m^2$')
sns.scatterplot(x=cube3df.errorAlong, y=cube3df.BSTAR, hue =cube3df.NORAD_CAT_ID, palette='Spectral',legend=None,ax= ax[0])
ax[2].set_title(r'Volume 0.003-1 $m^2$')
sns.scatterplot(x=bigvdf.errorAlong, y=bigvdf.BSTAR, hue =bigvdf.NORAD_CAT_ID,palette='Spectral',legend=None,ax= ax[1])
ax[0].set(xlabel='Cross track error [km]', ylabel='Drag coeffcient [1/re]')
ax[1].set(xlabel='Cross track error [km]', ylabel='Drag coeffcient [1/re]')
ax[2].set(xlabel='Cross track error [km]', ylabel='Drag coeffcient [1/re]')
fig.savefig("data/dragcross_norad.png")

In [ ]:
fig, ax = plt.subplots(1,3,figsize = (15,5))
fig.suptitle('Atmospheric Drag Factor against In track error ')
ax[0].set_title(r'Average cross sectional area <0.1 $m^2$')
sns.scatterplot(x=surface1df.errorAlong, y=surface1df.BSTAR, hue =surface1df.NORAD_CAT_ID, palette='Spectral',legend=None, ax= ax[0])
ax[1].set_title(r'Average cross sectional area 0.1-0.5 $m^2$')
sns.scatterplot(x=surface2df.errorAlong, y=surface2df.BSTAR, hue =surface2df.NORAD_CAT_ID, palette='Spectral',legend=None,ax= ax[1])
ax[2].set_title(r'Average cross sectional area 0.5-2.8 $m^2$')
sns.scatterplot(x=surface3df.errorAlong, y=surface3df.BSTAR, hue =surface3df.NORAD_CAT_ID,palette='Spectral',legend=None,ax= ax[2])
ax[0].set(xlabel='In track error [km]', ylabel=r'Atmospheric Drag Factor [$1/r_e$]')
ax[1].set(xlabel='In track error [km]', ylabel=r'Atmospheric Drag Factor [$1/r_e$]')
ax[2].set(xlabel='In track error [km]', ylabel=r'Atmospheric Drag Factor [$1/r_e$]')
fig.savefig("data/crosssurface.png")

In [ ]:
sns.scatterplot(x=df.errorAlong, y=df.Alt, hue =df.NORAD_CAT_ID,palette='Spectral',legend=None)


In [ ]:
fig, ax = plt.subplots(1,3,figsize = (15,5))
fig.suptitle('Altitude against In track error')
ax[0].set_title(r'Surface area <0.1 $m^2$')
sns.scatterplot(x=surface1df.errorAlong, y=surface1df.Alt, hue =surface1df.NORAD_CAT_ID, palette='Spectral',legend=None, ax= ax[2])
ax[1].set_title(r'Surface area 0.1-0.5 $m^2$')
sns.scatterplot(x=surface2df.errorAlong, y=surface2df.Alt, hue =surface2df.NORAD_CAT_ID, palette='Spectral',legend=None,ax= ax[0])
ax[2].set_title(r'Surface area 0.5-2.8 $m^2$')
sns.scatterplot(x=surface3df.errorAlong, y=surface3df.Alt, hue =surface3df.NORAD_CAT_ID,palette='Spectral',legend=None,ax= ax[1])
ax[0].set(xlabel='In track error [km]', ylabel=r'Altitude [m]')
ax[1].set(xlabel='In track error [km]', ylabel=r'Altitude [m]')
ax[2].set(xlabel='In track error [km]', ylabel=r'Altitude [m]')
fig.savefig("data/altsurfaceintrack.png")

fig, ax = plt.subplots(1,3,figsize = (15,5))
fig.suptitle('Atmospheric Drag Factor against In track error ')
ax[0].set_title(r'Surface area <0.1 $m^2$')
sns.scatterplot(x=df.ECCENTRICITY, y=df.errorAlong, hue =df.name, palette='Spectral',legend=None, ax= ax[0])
ax[1].set_title(r'Surface area 0.1-0.5 $m^2$')
sns.scatterplot(x=df.ECCENTRICITY, y=df.errorCross, hue =df.name, palette='Spectral',legend=None,ax= ax[1])
ax[2].set_title(r'Surface area 0.5-2.8 $m^2$')
sns.scatterplot(x=df.ECCENTRICITY, y=df.errorRadial, hue =df.name,palette='Spectral',legend=None,ax= ax[2])
ax[0].set(xlabel='Eccentricity', ylabel=r'Along track error [km]')
ax[1].set(xlabel='Eccentricity', ylabel=r'In track error [km]')
ax[2].set(xlabel='Eccentricity', ylabel=r'Radial error [km]')
fig.savefig("data/crosssurface.png")